In [ ]:
# Package Imports
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# Load Data and Create Sparse Matrix

In [ ]:
# Read books data
books = pd.read_csv('red_books.csv')
books

In [ ]:
# Read sparse matrix encoding into df
sparse_df = pd.read_csv('users_sparse.csv')
sparse_df

In [ ]:
# Create Scipy sparse matrix and convert to compressed sparse row format for operations
row = np.array(sparse_df.r_index)
col = np.array(sparse_df.c_index)
dat = np.array(sparse_df.data)
users = coo_matrix((dat,(row,col)),shape=(np.unique(row).size,np.unique(col).size))
users = users.tocsr()

# Determining k

In [ ]:
n_users = np.unique(row).size
n_users

In [ ]:
# From some source somewhere
k = (math.sqrt(n_users))/2
k = round(k)
print(k)

In [ ]:
# Systematic approach

# KNN for Cosine Similarities

In [ ]:
# Set up KNN
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=377)
knn.fit(users)

# Pick a user id from 0 to 567805
user_id = 223

# Get k neighbors for user_id
nbrs = knn.kneighbors(users.getrow(user_id))

In [ ]:
# Gives user index and distance (angle between vectors, 0 means cosine simlarity == 1)
print(nbrs)

# With Cosine Similarity Function Instead

In [ ]:
# Pick a user id from 0 to 567805
user_id = 223

# Calculate pairwise cosine similarities between user and all other users
similarities = cosine_similarity(users.getrow(user_id),users)
sim_index = pd.DataFrame({'ind':np.unique(row),'sim':similarities.flatten()})

#ind = np.argpartition(similarities, -k)[-k:]

# Weighted Average